# Se connecter au drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
%cd /mydrive/PLDAC/darknet
%ls

In [ ]:
!chmod +x ./darknet
%ls

# Fonctions d'affichage sur le drive

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
%cd ./cfg
!sed -i 's/batch=32/batch=1/' yolov4-custom.cfg
!sed -i 's/subdivisions=32/subdivisions=1/' yolov4-custom.cfg
%cd ..

# Test de prédiction sur l'image 0041408

In [ ]:
!./darknet detector test data/piford.data cfg/yolov4-custom.cfg backup/yolov4-custom_best.weights data/piford/0041408.bmp -thresh 0.3 -dont_show -map -ext_output -out 0041408.txt

In [ ]:
imShow('predictions.jpg')

# Prédictions avec evaluation.py



In [ ]:
import evaluation 

## Cas d'une prédiction pour un modèle dont les poids sont dans backup

Cette première utilisaiton de evaluation.py correspond par exemple à un cas où l'on vient d'entraîner un modèle et que l'on souhaite désormais effectuer toutes les prédictions sur les images de test ainsi que d'évaluer notre modèle.   
On obtient alors les résultats des prédictions dans le dossier 'prediction' et toutes les statistiques d'évaluation dans le dossier 'prediction_stats'. Ces deux dossiers se trouvent dans le dossier darknet.    
Dans le dossier 'prediction' on trouve pour chaque image de test : 
  - Un fichier .txt avec le résultat de la prédiction.
  - Une image au format .jpg avec la bbox réelle et celle prédite ainsi que le score de confiance et le score IoU.     
  
Dans le dossier 'prediction_stats' on trouve :
  - avg_iou.txt : contenant la moyenne des scores IoU obtenus lors des prédictions.
  - best_score_map.txt : contenant le meilleur score MAP obtenu (selon le threshold utilisé pour effectuer la prédiction).
  - best_threshold.txt : contenant le threshold permettant de maximiser la MAP. 
  - chart.png : correspondant à la courbe d'apprentissage qui avait été obtenue lors du dernier train effectué.
  - confidences.txt : contenant les scores de confiance pour chaque image.
  - confusion_matrix.jpg : la matrice de confusion obtenue sur nos données test (vrai positif, vrai négatig, faux positif et faux négatif).
  - coord_pred.txt : contenant les coordonnées prédites pour chaque image. 
  - coord_true.txt : contenant les coordonées réelles pour chaque image. 
  - MAP_vs_threshold.jpg : courbe présentant l'évolution de la MAP en fonction du threshold. 
  - Precisions_curve.jpg : courbe présentant l'évolution de la précision en fonction du threshold. 
  - Precisions_recall_curve : courbe présentant la précision en fonction du rappel. 
  - prediction_times.txt : contenant les temps de chaque prédiction ainsi que le temps moyen de prédiction (en ms et FPS).
  - Recalls_curve.jpg : courbe présentant l'évolution du recall en foncion du threshold. 
  - scores_ciou.txt : contenant les score CIoU obtenus pour chaque image. 
  - test.txt : fichier listant les images ayant été utilisées pour le test.
  - train.txt : fichier listant les images ayant été utilisées pour l'entraînement.
  - yolov4-custom_best.weights : meilleur poids ayant été obtenu lors de l'appprentissage. 



In [ ]:
evaluation.run('data/test.txt', '', '', '', True)


## Cas pour évaluer tous les modèles 

Ce second cas de figure permet de

In [ ]:
evaluation.run_all('data/test.txt')